# Daniel Bairamian 260669560 
## ECSE 415 Computer Vision
### Assignment 3 Part One

# Segmentation - K-means clustering and Expectation Maximization

##  K-means

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import copy

from skimage import color
from random import randint
from scipy.stats import norm
from scipy.stats import multivariate_normal


In [ ]:
# Some constants
EPSILON = 1e-8 # Used to prevent divisions by 0

In [ ]:
# Flattens a 2D image into a 1D image
# Takes a 3D numpy array and returns a 2D numpy array
def flatten_image(img):
    np.testing.assert_equal(len(img.shape), 3)
    return img.reshape(img.shape[0]*img.shape[1],img.shape[2])

In [ ]:
# Helper function to show images
def show_images(images, cols , titles):
    
    n_images = len(images)
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        plt.xticks([]), plt.yticks([])
        plt.imshow(image, cmap="gray")
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [ ]:
# K-Color an image
# loop through the image twice to get the average color of a label
# then reapply the average color of the label to the class
def K_color_image(img, labs, k):
    
    original_shape = img.shape
    temp_image = copy.deepcopy(img)
    temp_label = copy.deepcopy(labs)
    img_flat = flatten_image(temp_image)
    
    # make this function work in both RGB and Greyscale
    avg_color   = np.zeros([k, img.shape[2]])
    class_count = np.zeros([k, 1]) 
    
    
    for i, (img_px, label) in enumerate (zip(img_flat, temp_label)):
        # moving average of label colour
        avg_color[label]  = np.divide( (avg_color[label]*class_count[label]) + img_px,  class_count[label] + 1) 
        class_count[label]+= 1
    
    for i, (img_px, label) in enumerate (zip(img_flat, temp_label)):
        img_flat[i] = avg_color[label]
    
    
    return img_flat.reshape(original_shape[0], original_shape[1], original_shape[2])

In [ ]:
def k_means_cluster(data, k, iter_count):
    
    labels_list = []
    
    # Randomly assign a label
    labels = np.random.randint(low=0, high=k, size=data.shape[0])
    
    # Randomly assign centers between minimum and maximum values of the data
    centers = np.random.uniform(low=0., high=1., size=(k, data.shape[1]))
    centers = centers * (np.max(data, axis=0) - np.min(data, axis=0)) + np.min(data, axis=0)

    for i in range(0, iter_count):
        # save the first 5 iteration labels
        if i < 5:
            labels_list.append(labels)
        # compute the distances
        distances = np.array([np.linalg.norm(data - cen, axis=1) for cen in centers])
        # assign the label to the closest distance
        lables_new = np.argmin(distances, axis=0)

        if (labels == lables_new).all():
            # If labels didn't change, we're done, break
            labels = lables_new
            break
        else:
            labels = lables_new
            for c in range(k):
                # prevent taking the mean of empty classes
                # occurs when a class doesn't have any labels associated with it
                # in that case don't update the center since there are non
                # this means the center isn't updated, but is still there
                
                # maybe a more accurate approach would be to tag the class that doesn't have points associated to it
                # and simply remove that class (i.e. decrement K)
                # but this seems to be working fine :) 
                if(len(data[labels==c])!=0):
                    centers[c] = np.mean(data[labels == c], axis=0)
                else:
                    continue
    
    # save the last label
    labels_list.append(labels)
    return labels_list

In [ ]:
def show_result(image, k, label_array):
    
    titles = []
    images = []
    labels = []
    
    for i, label_list in enumerate(label_array):
        seg_output = K_color_image(image, label_list, k)
        images.append(seg_output)
        if i != len(label_array)-1:
            titles.append("Iteration: " + str(i+1))
        else:
            titles.append("Final Iteration")
        
    show_images(images, len(images), titles)

In [ ]:
# load image
img        = cv2.imread("flower.jpg")
img_rgb    = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_gscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# repeat grayscale image 3 times to make it a 3 dimentional matrix
# doing so it uses the same code as the RGB image for K means and K colouring
img_gscale_reshaped = np.repeat(img_gscale[:, :, np.newaxis], 3, axis=2)

original_images = [img_rgb, img_gscale, img_gscale_reshaped]
original_titles = ["Original Image RGB", "Original Image Gray Scale", "Gray Scale Reshaped"]

show_images(original_images, len(original_images)/3, original_titles)

In [ ]:
# Flatten input images
img_rgb_flat = flatten_image(img_rgb)
img_gs_flat = flatten_image(img_gscale_reshaped)

### RGB Image K = 2

In [ ]:
k      = 2
n_iter = 1000

K_means_result = k_means_cluster(img_rgb_flat, k, n_iter)
show_result(img_rgb, k, K_means_result)

### RGB Image K = 3

In [ ]:
k      = 3
n_iter = 1000

K_means_result = k_means_cluster(img_rgb_flat, k, n_iter)
show_result(img_rgb, k, K_means_result)

### Grayscale Image K = 2

In [ ]:
k      = 2
n_iter = 1000

K_means_result = k_means_cluster(img_gs_flat, k, n_iter)
show_result(img_gscale_reshaped, k, K_means_result)

### Grayscale Image K = 3

In [ ]:
k      = 3
n_iter = 1000

K_means_result = k_means_cluster(img_gs_flat, k, n_iter)
show_result(img_gscale_reshaped, k, K_means_result)

### The segmentation map of the RGB image was better than the grayscaled version. This is because the grayscale image has strictly less information to work with than the RGB image. The RGB image has 3 dimensions to use as information to determine which cluster a data point belongs to, whereas the grayscale image only has 1. 

# Expectation Maximization

In [ ]:
def E_Step(k, img, clusters_alpha, clusters_mean, clusters_std, verbose):
   
    # get the height and width of the image
    height, width, _ = img.shape
    
    # initalize an empty list of clusters
    clusters = np.zeros((height, width), np.int64)
    # initialize an empty list of multivariate gaussians
    cluster_gaussians_list = []
    
    # create gaussians for each class
    for i in range(0, k):
        try:
            cluster_gaussians_list.append( multivariate_normal(clusters_mean[i], clusters_std[i]) )
        except:
            # the multivariate normal sometimes fails to execute. the standard deviation matrix is singular
            # I believe this happens when a class cannot have points associated to, and so it becomes empty
            # and since no points are associated to it, the matrix becomes 0. And Zero matrices are not invertible hence singular
            
            # To deal with that, i just set the mean to be 0, and the standard deviation to be all ones.
            # This might not be best approach to this problem but at least this makes it so the code doesn't crash
            
            # A better alternative would probably be to tag the class that doesn't have labels associated to it 
            # So we can exclude it from any computation so long as it doesn't have labels associated to it
            # but honestly this would take too much time for little to no added value.
            
            # Note: Instead of making the mean 0, i instead make it an array of EPSILONs, which is a tiny constant i defined on top
            # this is just so that no divisions by zeros occur
            if verbose:
                print("Cannot calculate Multivariate gaussian for class: ", k)
                
            cluster_gaussians_list.append(multivariate_normal( np.ones(clusters_mean[i].shape, np.int64)*EPSILON, np.ones(clusters_std[i].shape, np.int64)) )   
    
    for h in range(0, height):
        for w in range(0, width):
            
            proba_results = []
            proba_sum = 0
            # calculate the probability of each cluster
            for i in range (0, k):
                gauss_k = clusters_alpha[i] * cluster_gaussians_list[i].pdf(img[h, w, :])
                proba_results.append(gauss_k)
                proba_sum += gauss_k
                
            for i in range (0, k):
                # Added epsilon to prevent division by zero when class doesn't have labels
                proba_results[i] /= proba_sum + EPSILON
            # assign a cluster to each point according to the calculated probabilty
            clusters[h, w] = np.argmax(proba_results)
            
    return clusters

In [ ]:
def M_Step(k, img, clusters):
    
    # get height, width and depth of the image
    height, width, depth = img.shape
    
    # initialize empty arrays of updated arrays
    clusters_alpha_new = np.zeros((k), np.float64)
    clusters_mean_new = np.zeros((k, depth), np.float64)
    clusters_std_new = np.zeros((k, depth), np.float64)

    # Compute the clusters alpha and mean
    for h in range(0, height):
        for w in range(0, width):
            clusters_alpha_new[clusters[h, w]] += 1
            clusters_mean_new[clusters[h, w], :] += img[h, w, :]


    # Compute the clusters standard deviation
    for i in range(k):
        # Added epsilon to prevent division by zero when class doesn't have labels
        clusters_mean_new[i] /= clusters_alpha_new[i] + EPSILON
        for h in range(height):
            for w in range(width):
                if(clusters[h, w] == i):
                    clusters_std_new[i] +=  np.dot(img[h, w, :] - clusters_mean_new[i, :], (img[h, w, :] - clusters_mean_new[i, :]).T)
                    

    for i in range(k):
        # Added epsilon to prevent division by zero when class doesn't have labels
        clusters_std_new[i] /= (clusters_alpha_new[i] - 1) + EPSILON 
        clusters_std_new[i] = np.sqrt(clusters_std_new[i])
    clusters_alpha_new /= (height * width)
    
    return clusters_alpha_new, clusters_mean_new, clusters_std_new

In [ ]:
def EM_Initialize(k, img):
    
    # get height, width, dpeth
    height, width, depth = img.shape
    
    # initialize empty arrays for 
    # alpha, mean and standard deviation
    clusters = np.zeros((height, width), np.int64)
    clusters_alpha = np.zeros((k), np.float64)
    clusters_mean = np.zeros((k, depth), np.float64)
    clusters_std = np.zeros((k, depth), np.float64)
    
    # give a random cluster for each point
    for h in range(0, height):
        for w in range(0, width):
            cluster = np.random.randint(k, size=1)
            cluster = cluster[0]
            clusters[h, w] = cluster
            clusters_alpha[cluster] += 1
            clusters_mean[cluster, :] += img[h, w, :]        

    # compute the standard deviation
    for i in range(0, k):
         # compute the mean
        clusters_mean[i] /= clusters_alpha[i]
        for h in range(0, height):
            for w in range(0, width):
                if(clusters[h, w] == i):
                    clusters_std[i] +=  np.dot(img[h, w, :] - clusters_mean[i, :], (img[h, w, :] - clusters_mean[i, :]).T)

    for i in range(0, k):
        clusters_std[i] /= (clusters_alpha[i] - 1) + EPSILON 
        clusters_std[i] = np.sqrt(clusters_std[i])
    clusters_alpha /= (height * width) + EPSILON

    return clusters_alpha, clusters_mean, clusters_std, clusters

In [ ]:
def EM(k, img, iter_count, verbose = False):
    total_labels = []
    # initialize all the data we need
    clusters_alpha,  clusters_mean, clusters_std, clusters = EM_Initialize(k, img)
    
    # save the first 5 iterations
    for i in range(iter_count):
        if i < 5:
            total_labels.append(clusters.flatten())
        if verbose:
            print("Iteration: ", i+1, "/", iter_count)
        
        # E step
        clusters = E_Step(k, img, clusters_alpha, clusters_mean, clusters_std, verbose)
        # M step
        clusters_alpha, clusters_mean, clusters_std = M_Step(k, img, clusters)
    
    # save the last iteration
    total_labels.append(clusters.flatten())
    return total_labels

### RGB Image K = 2

In [ ]:
k      = 2
n_iter = 15

EM_results = EM(k, img_rgb.copy(), n_iter, verbose=True)
show_result(img_rgb, k, EM_results)

### RGB Image K = 3

In [ ]:
k      = 3
n_iter = 15

EM_results = EM(k, img_rgb.copy(), n_iter, verbose=True)
show_result(img_rgb, k, EM_results)

### Grayscale Image K = 2

In [ ]:
k      = 2
n_iter = 15

EM_results = EM(k, img_gscale_reshaped.copy(), n_iter, verbose=True)
show_result(img_gscale_reshaped, k, EM_results)

### Grayscale Image K = 3

In [ ]:
k      = 3
n_iter = 15

EM_results = EM(k, img_gscale_reshaped.copy(), n_iter, verbose=True)
show_result(img_gscale_reshaped, k, EM_results)

### Just like the K means algorithm, the EM segmentation also performs better on the RGB image than on the grayscale image, and this for the same reasons stated above.

### An additional issue the grayscale version faces that is less likely to occur in the RGB version is singular covariance matrices. Since there is less information to work with, the algorithm might converge to a steady state where it is only able to assign points to C classes, where C is strictly less than the original K picked. (C < K)

### This then means that during the training loop, the covariance matrix and mean array for some class will become 0, which could cause issues if it is not properly dealt with. (matrix cannot be inverted).

### This issue can also occur in the RGB case, but since we have more information to work with, this occurance of a class being empty is less likely

# Datapoints Test

In [ ]:
data = np.float32(
    np.vstack((
        np.random.normal(
            loc = np.array([5,5]),
            scale = np.array([3,2]),
            size = (1000, 2)),
        np.random.normal(
            loc = np.array([-5, -5]),
            scale = np.array([5, 1]),
            size = (1000, 2)
        ))))


# EM expects 2D data, so just giving data arbitrairy rectangular shape
# the reshape shouldn't affect the results since there are no Kernels used
# and no convolutions. Meaning there is no spatial information being taken into consideration when segmenting
# which is why this reshape shouldn't affect the final segmentation, it just makes my life easier :)

data2 = data.reshape(40, 50, 2)

### K Means 

In [ ]:
iter_count = 100000
k = 2

label_cluster_KM = k_means_cluster(data, k, iter_count)

In [ ]:
# can easily split the data into different clusters depending on their labels
A = data[label_cluster_KM[-1].ravel()==0]
B = data[label_cluster_KM[-1].ravel()==1]

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(data[:,0], data[:,1],"ko")
plt.title("Data points")
plt.subplot(122)
plt.scatter(A[:,0], A[:,1], color='b')
plt.scatter(B[:,0], B[:,1], color='r')

plt.title("K Means: Data points - Labeled")
plt.show()

### EM

In [ ]:
iter_count = 200
k = 2

labels_EM = EM(k, data2.copy(), iter_count)

In [ ]:
# can easily split the data into different clusters depending on their labels
A = data[labels_EM[-1].flatten().ravel()==0]
B = data[labels_EM[-1].flatten().ravel()==1]

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(data[:,0], data[:,1],"ko")
plt.title("Data points")
plt.subplot(122)
plt.scatter(A[:,0], A[:,1], color='b')
plt.scatter(B[:,0], B[:,1], color='r')

plt.title("EM: Data points - Labeled")
plt.show()

### From the results above, we can see that EM performs better than K means at classifyingour generated data. This is to be expected since the data points were generated with 2-D gaussians. 

### The reason why this is to be expected, is because K means uses eucledian distance to jugde class membership of a datapoint. However, it does not take variance into consideration. Even if a point is closer to a center in eucledian distance, it may be very unlikely to actually belong to that center's class, since the variance of that class is very small in the direction of that datapoint

### EM is better at classifying our gaussian data points, because it uses probabilities rather than eucledian distance to judge class membership. In layman's terms, it uses probabilities to "warp" the space, so that class centers appear closer or further than they would in eucledian space. 

# Normalized graph-cut and Mean-Shift segmentation

## Normalized Graph Cut

In [ ]:
from skimage import segmentation, color
from skimage.future import graph
from skimage.segmentation import quickshift

In [ ]:
# load image
img        = cv2.imread("flower.jpg")
img_rgb    = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

original_images = [img_rgb]
original_titles = ["Original Image RGB"]

show_images(original_images, len(original_images), original_titles)

In [ ]:
images = []
titles = []

compactness_values = [1, 10, 50, 100]
n_segments_values  = [50, 200, 500]
thresh_values      = [0.005, 0.05, 0.5]

iter_count = 0
total_iters= len(compactness_values)*len(n_segments_values)*len(thresh_values)

for comp in compactness_values:
    for seg in n_segments_values:
        for threshold in thresh_values:
            iter_count += 1
            print("Iteration: ", iter_count, "/", total_iters)
            # K means segmenration to generate super pixels
            labels1 = segmentation.slic(img_rgb, compactness=comp, n_segments=seg)
            # Region adjacency graph
            g = graph.rag_mean_color(img_rgb, labels1, mode='similarity')
            labels2 = graph.cut_normalized(labels1, g, thresh=threshold)
            # apply graph cut
            out2 = color.label2rgb(labels2, img_rgb, kind='avg')
    
            # append output to result list
            images.append(out2)
            titles.append("Compactness: "+ str(comp)+ ", N-Segments: " + str(seg) +", Trheshold: " + str(threshold)) 

In [ ]:
show_images(images, len(images), titles)

### The n_segments parameter seem to affect how many classes we think there are, similar to K in our K means and EM.

### Compactness seems to have an effect on depth, and the amount of details preserved in the background. A higher value of compactness seems to give more importance to the background, and lower values seem to give less importance to the background.


### The threshold parameter seems to have an effect on the sub-division of a class. The value of the threshold has an effect on how the region within a class is divided. 

## Mean Shift Segmentation

In [ ]:
images = []
titles = []

kernel_values    = [5, 15, 30, 60]
max_dist_values  = [5, 20, 40, 60, 100]

iter_count = 0
total_iters= len(kernel_values)*len(max_dist_values)

for kernel in kernel_values:
    for dist in max_dist_values:
        iter_count += 1
        print("Iteration: ", iter_count, "/", total_iters)
        # apply meanshift
        label = quickshift(img_rgb, kernel_size=kernel, max_dist=dist)
        out = color.label2rgb(label, img_rgb, kind='avg')
        images.append(out)
        titles.append("Kernel: "+ str(kernel)+ ", Distance: " + str(dist)) 

In [ ]:
show_images(images, len(images), titles)

### The kernel size seems to have an effect on how many different clusters we have. It controls the size of a kernel, which then must mean that the higher this kernel, the more information it treats at a given time, so the *fewer* the classes we will have. Essentially this parameter is like K in K means and EM, only it seems to inversely proportional in this case. (The higher the number the fewer the classes)


### The max distance also seems to have an effect on how many different clusters we have. It must be analogous to the K means distance, where it  represent some threshold distance to determine if a datapoint is a member of a class